In [2]:
import pygame, types, threading, traceback

## Hey, let's code a better game!
OK, I'm all for it. But first, we need to modify the main class in order to be able to handle events. Here is a way to do it:


In [3]:
class Game:
    def __init__(self):
        self.size = (500,500)
        self.running = True
        self.scene = list()
        self.event_handlers = dict()
        self.event_handlers[(('type',pygame.QUIT),)] = self.on_quit
        self.event_handlers[(('type',pygame.KEYDOWN), ('key',pygame.K_q))] = self.on_quit
        self.event_handlers[(('type',pygame.KEYDOWN), ('key',pygame.K_ESCAPE))] = self.on_quit
        self.flipdelay=16
        self.tickcounter=0
        
    def render(self):
        self.disp.fill((0,0,0))
        for obj in self.scene:
            try:
                obj.render(self.disp)
            except Exception:
                traceback.print_exc()
                self.scene.remove(obj)
                print("Exception during render: Object "+str(obj)+" removed from the scene")
        pygame.display.flip()

    def update(self):
            dt=pygame.time.get_ticks()- self.tickcounter
            for obj in self.scene:
                try:
                    obj.update(dt)
                except Exception:
                    traceback.print_exc()
                    self.scene.remove(obj)
                    print("Exception during update: Object "+str(obj)+" removed from the scene")
            self.tickcounter=pygame.time.get_ticks()
            pygame.time.delay(self.flipdelay)
        

        
    def on_quit(self, event):
        self.running = False
        
    def process_events(self):
        for event in pygame.event.get():
            dire = dir(event)
            for eh in self.event_handlers.keys():
                callit=True
                for (attrname,attrvalue) in eh:
                    if (not attrname in dire) or (event.__getattribute__(attrname)!=attrvalue):
                        callit=False
                        break
                if callit:
                    self.event_handlers[eh](event)
        
    def mainloop(self):
        pygame.init()
        self.disp=pygame.display.set_mode(self.size, pygame.HWSURFACE | pygame.DOUBLEBUF)
        self.tickcounter=pygame.time.get_ticks()
        while( self.running ):
            try:
                self.render()
                self.process_events()
                self.update()
            except Exception:
                traceback.print_exc()
                pygame.time.delay(10000)
        pygame.quit()

OK, maybe I should explain a little there. We want a way to attribute a function to each possible event that the program may receive. Typically in a program that is where you would have a lot of if: elif: blocks. Here we want to be able to modify this attribution dynamically.

So we make a dictionnary: events as keys and functions as values. The problem is that different events will have different attributes. The solution here is to use python's introspection abilities. We make a list of size 2 tuples that describe an attribute name and a value and we use the function __getattribute__ in order to check its value at runtime.

If an entry of the dictionnary contains only criterions that the current event passes, its associated handler function is called, passing the event as an argument. That should cover all the cases we need.

I also added some exception handling code so that when an object I added triggers an exception, instead of making the whole application crash, it just removes the object from the scene.


In [4]:
game = Game()
th = threading.Thread(target = game.mainloop)
th.start()

All right, let's roll. I want to make a zelda-like game. Last time I attempted that with pygame, I remember that it was really slow to redisplay  the whole map every frame. Let's check that. I have a grass tile I can use (courtesy of Zabin, Hyptosis, and Danial Cook who put [this tileset](https://opengameart.org/content/castle-tiles-for-rpgs) in an open source license) :
<img src="art/castle/grass.png">

In [5]:
class Map:
    def __init__(self):
        # We will need to load some images and make a better Tile class later
        # but now I just want to test the speed of the blit operation
        self.tile = pygame.image.load('art/castle/grass.png')
        self.avg_time=0.0
        
    def update(self, dt):
        return
    
    def render(self, disp):
        ta = pygame.time.get_ticks()
        for y in range(0,500,32):
            for x in range(0,500,32):
                disp.blit(self.tile, (x,y))
        self.avg_time = 0.9*self.avg_time + 0.1*float(pygame.time.get_ticks()-ta)
        #print(pygame.time.get_ticks()-ta)
        

In [6]:
# Tip: when trying different version of an object, just reset the scene list by uncommenting this:
# game.scene = list()

game.scene.append(Map())

Perfect.


In [7]:
game.scene[0].avg_time

6.740605672913669

Around 4 ms. That's actually pretty bad, but let's pretend it is ok for now.
Now we have to add a character: <IMG src="art/LPC/walk.png">

In the pixel world, clothes are optional. I mean, seriously, they come as separate sprites so that the character can be fully customizable. This is a series of 64x64 sprites. Let's load and display a sequence.

In [8]:
class Character:
    def __init__(self, x, y):
        self.img=pygame.image.load("art/LPC/walk.png")
        self.frames = list()
        self.cycle_index = 0
        self.cycle_tick = 0
        self.cycle_tick_per_frame = 100
        self.cycle_length = 7
        for i in range(self.cycle_length):
            self.frames.append(self.img.subsurface((64+i*64,0,64,64)))
        self.pos = (x,y)
        
    def update(self, dt):
        self.cycle_tick = (self.cycle_tick + dt) % (self.cycle_length*self.cycle_tick_per_frame)
        self.cycle_index = int(self.cycle_tick/self.cycle_tick_per_frame)
        pass
        
    def render(self, display):
        display.blit(self.frames[self.cycle_index], (self.pos[0]+200, self.pos[1]))


In [10]:
game.scene.append(Character(100,100))

Nice! Now we have an animated sprite in front of a map. Let's make it a bit smarter by loading all the 4 animations plus the 4 still frames. Then we will trigger animations with the key presses.

In [11]:
class Character:
    def __init__(self, x, y):    
        self.img=pygame.image.load("art/LPC/walk.png")
        # Each animation is stored in the anim dict as a list with the following 
        # format: (tick_per_frame, frame1, frame2, ...)
        
        self.anim = dict()
        self.cycle_index = 0
        self.cycle_tick = 0
        seq = list()
        seq.append(80) # ticks per frame
        for i in range(8):
            seq.append(self.img.subsurface((64+i*64,0,64,64)))
        self.anim["up"] = seq
        
        seq = list()
        seq.append(80) # ticks per frame
        for i in range(8):
            seq.append(self.img.subsurface((64+i*64,128,64,64)))
        self.anim["down"] = seq

        seq = list()
        seq.append(80) # ticks per frame
        for i in range(8):
            seq.append(self.img.subsurface((64+i*64,64,64,64)))
        self.anim["left"] = seq

        seq = list()
        seq.append(80) # ticks per frame
        for i in range(8):
            seq.append(self.img.subsurface((64+i*64,192,64,64)))
        self.anim["right"] = seq

        self.current_anim = "up"
        self.current_frames = self.anim[self.current_anim]
        self.pos = [x,y]

    def update(self, dt):
        ca = self.anim[self.current_anim]
        self.cycle_tick = (self.cycle_tick + dt) % ((len(ca)-1)*ca[0])
        self.cycle_index = int(self.cycle_tick/ca[0])
        pass
        
    def render(self, display):
        ca = self.anim[self.current_anim]
        display.blit(ca[1+self.cycle_index], (self.pos))
        

In [12]:
game.scene.remove(game.scene[1])
game.scene.append(Character(100,100))

Good. We can test that we can change the sequence with:

In [13]:
game.scene[1].current_anim = "left"

Now we need to control the sequence with key presses. We do that by adding a handler in the Character class and by registering it in the handlers dict in the game class. 



In [14]:
def handle_keydown(self, evt):
    if evt.key == pygame.K_LEFT:
        self.current_anim="left"
    elif evt.key == pygame.K_RIGHT:
        self.current_anim="right"
    if evt.key == pygame.K_UP:
        self.current_anim="up"
    if evt.key == pygame.K_DOWN:
        self.current_anim="down"
    
Character.handle_keydown = handle_keydown



Our handler will be a method of the class Character, so we need to also give an instance of the class to the handler registration. It is time to make an instance of Character a bit special and keep track of it as a member of the class game.

In [15]:
game.scene.remove(game.scene[1])
game.player = Character(100,100)
game.scene.append(game.player)

game.event_handlers[(('type',pygame.KEYDOWN),)] = game.player.handle_keydown


It feels like we are going to need to refresh the whole scene from time to time. Better add a function to the game class to do that.

In [16]:
def refresh_scene(self):
    self.scene=list()
    self.player = Character(100,100)
    self.scene.append(Map())
    self.scene.append(self.player)
    self.event_handlers[(('type',pygame.KEYDOWN),)] = self.player.handle_keydown
    
Game.refresh_scene = refresh_scene
game.refresh_scene = types.MethodType(refresh_scene, game)

In [17]:
game.refresh_scene()

Now we want the character to move in the direction it is going. For that we will replace its update function:

In [18]:
def update(self, dt):
    dirs={"up":(0,-0.5),"down":(0,0.5), "right":(1,0), "left":(-1,0)}
    ca = self.anim[self.current_anim]
    dxdy = dirs[self.current_anim]
    self.cycle_tick = (self.cycle_tick + dt) % ((len(ca)-1)*ca[0])
    self.cycle_index = int(self.cycle_tick/ca[0])
    self.pos[0]+=dxdy[0]
    self.pos[1]+=dxdy[1]
    
Character.update = update
game.refresh_scene()

Et voila. That's all for tonight.